# W5-CRISPR-Cas9-inframe_deletion

In Workflow 5, CRISPR-Cas9 combines ssDNA bridging and Gibson cloning to create in-frame deletions (Figure 6A). This ensures that the downstream genes are not disrupted, which is crucial for functional studies of gene knockouts. This workflow can be used to study gene function by knocking out specific genes in streptomycetes and observing the resulting phenotypic changes. It can also be used to create markerless deletions, which are important for constructing clean genetic backgrounds for further genetic manipulations. To get started, download the pCRISPR-Cas9 file and a genome file (e.g., S. coelicolor A3), then select the genes to delete. StreptoAIM will generate the necessary primers and plasmid constructs (S5).


In [1]:
import sys
import os
from Bio.Restriction import StuI
from pydna.dseqrecord import Dseqrecord
import pandas as pd
from datetime import datetime


# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from streptocad.sequence_loading.sequence_loading import (
    load_and_process_plasmid,
    load_and_process_genome_sequences
)
from streptocad.utils import polymerase_dict, create_primer_df_from_dict,generate_project_directory_structure
from streptocad.primers.primer_generation import create_idt_order_dataframe
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging,make_ssDNA_oligos
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.gibson_cloning import (
    find_up_dw_repair_templates,
    assemble_multiple_plasmids_with_repair_templates_for_deletion,
    update_primer_names
)
from streptocad.primers.primer_generation import create_dseqrecords_from_df,checking_primers, primers_to_IDT


## INPUT

In [2]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../data/plasmids/pCRISPR–Cas9_plasmid_addgene.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
genes_to_KO = ['SCO5087', 'SCO5089', 'SCO5090']

#### Advanced settings ####

# 4 Filtering metrics for sgRNAs
gc_upper = 0.72
gc_lower = 0.2
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 2

# 5 Choose polymerase and target melting temperature
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']
melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 23
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('CGGTTGGTAGGATCGACGGC')
dw_homology = Dseqrecord('GTTTTAGAGCTAGAAATAGC')

# Computation

In [3]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(path_to_genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9'
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were outside the designated border in SCO5087. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5087. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5087. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5089. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5089. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5090. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5090. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5090. To incorporate this extent borders. Skipping to 

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
336,Streptomyces_coelicolor_A3_chromosome,SCO5090,5532706,1,-1,32,0.65,CGG,TGGACCACCCGGTGTTCTCC,CCCGGTGTTCTCC,0
332,Streptomyces_coelicolor_A3_chromosome,SCO5089,5532449,1,1,61,0.70,CGG,CGTGGAGTGCGCCGGTGAGA,TGCGCCGGTGAGA,0
331,Streptomyces_coelicolor_A3_chromosome,SCO5089,5532449,1,1,65,0.70,CGG,GAGTGCGCCGGTGAGACGGA,CCGGTGAGACGGA,0
330,Streptomyces_coelicolor_A3_chromosome,SCO5089,5532449,1,1,66,0.70,GGG,AGTGCGCCGGTGAGACGGAC,CGGTGAGACGGAC,0
328,Streptomyces_coelicolor_A3_chromosome,SCO5089,5532449,1,1,80,0.70,CGG,ACGGACGGGACGGACCTGTC,GGACGGACCTGTC,0
...,...,...,...,...,...,...,...,...,...,...,...
406,Streptomyces_coelicolor_A3_chromosome,SCO5090,5532706,1,-1,691,0.70,TGG,GCCGGATCGAACGGGTGGTG,CGAACGGGTGGTG,2
78,Streptomyces_coelicolor_A3_chromosome,SCO5087,5529801,1,-1,745,0.65,GGG,CGACGACGATCGGGGTGATC,GATCGGGGTGATC,2
452,Streptomyces_coelicolor_A3_chromosome,SCO5090,5532706,1,1,890,0.65,CGG,ACCCATCTGCGCGAGGTGCT,TGCGCGAGGTGCT,3
192,Streptomyces_coelicolor_A3_chromosome,SCO5087,5529801,1,1,838,0.70,TGG,TACGCGGGACGGCTTCGTGC,GACGGCTTCGTGC,3


In [4]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
336,Streptomyces_coelicolor_A3_chromosome,SCO5090,5532706,1,-1,32,0.65,CGG,TGGACCACCCGGTGTTCTCC,CCCGGTGTTCTCC,0
332,Streptomyces_coelicolor_A3_chromosome,SCO5089,5532449,1,1,61,0.70,CGG,CGTGGAGTGCGCCGGTGAGA,TGCGCCGGTGAGA,0
331,Streptomyces_coelicolor_A3_chromosome,SCO5089,5532449,1,1,65,0.70,CGG,GAGTGCGCCGGTGAGACGGA,CCGGTGAGACGGA,0
347,Streptomyces_coelicolor_A3_chromosome,SCO5090,5532706,1,-1,123,0.60,CGG,GTGCACGGTGGGTTCGAAGA,GTGGGTTCGAAGA,0
262,Streptomyces_coelicolor_A3_chromosome,SCO5087,5529801,1,1,271,0.70,AGG,CCCCTCGCCCTACCGTTCAC,CCCTACCGTTCAC,0
261,Streptomyces_coelicolor_A3_chromosome,SCO5087,5529801,1,1,277,0.65,CGG,GCCCTACCGTTCACAGGTCG,CGTTCACAGGTCG,0


## Output

In [5]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)
print(list_of_ssDNAs[0].name)

# cut plasmid
from Bio.Restriction import NcoI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI), key=lambda x: len(x), reverse=True)[0]
#print(linearized_plasmid)

sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)
sgRNA_vectors

SCO5090_loc_32


[Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279)]

In [6]:

# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"CRISPR-Cas9_plasmid_targeting_{row['locus_tag']}_position_{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}_{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  
    sgRNA_vectors[i].description = f'CRISPR-Cas9 targeting {", ".join(genes_to_KO)} for single gene knockout, assembled using StreptoAIM.'


In [7]:
print_plasmids = False

if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR-Cas9/{vector.id}.gb")

### IDT primers

In [8]:
idt_primers=primers_to_IDT(list_of_ssDNAs)
idt_primers

,Name,Sequence,Concentration,Purification
0,SCO5090_loc_32,CGGTTGGTAGGATCGACGGCTGGACCACCCGGTGTTCTCCGTTTTA...,25nm,STD
1,SCO5089_loc_61,CGGTTGGTAGGATCGACGGCCGTGGAGTGCGCCGGTGAGAGTTTTA...,25nm,STD
2,SCO5089_loc_65,CGGTTGGTAGGATCGACGGCGAGTGCGCCGGTGAGACGGAGTTTTA...,25nm,STD
3,SCO5090_loc_123,CGGTTGGTAGGATCGACGGCGTGCACGGTGGGTTCGAAGAGTTTTA...,25nm,STD
4,SCO5087_loc_271,CGGTTGGTAGGATCGACGGCCCCCTCGCCCTACCGTTCACGTTTTA...,25nm,STD
5,SCO5087_loc_277,CGGTTGGTAGGATCGACGGCGCCCTACCGTTCACAGGTCGGTTTTA...,25nm,STD


# In frame deletion

In [9]:
# Make repair templates
repair_templates_data = find_up_dw_repair_templates(genome, 
                                                    genes_to_KO, 
                                                    target_tm=melting_temperature, 
                                                    
                                                    primer_tm_kwargs={'conc':primer_concentration, 'prodcode':chosen_polymerase} )

repair_templates_data

[{'name': 'SCO5087',
  'up_repair': Amplicon(1000),
  'up_forwar_p': f1000 16-mer:5'-GACGCCGGAAATCCAG-3',
  'up_reverse_p': r1000 16-mer:5'-CCCATCTCCCTTCGAC-3',
  'tm_up_forwar_p': 56,
  'tm_up_reverse_p': 54,
  'ta_up': 58,
  'location_up_start': 5528800,
  'location_up_end': 5529800,
  'dw_repair': Amplicon(1000),
  'dw_forwar_p': f1000 16-mer:5'-GCGTCCTCATCACCGG-3',
  'dw_reverse_p': r1000 15-mer:5'-GAGTAGAGGCGGCCC-3',
  'tm_dw_forwar_p': 58,
  'tm_dw_reverse_p': 57,
  'ta_dw': 61,
  'location_dw_start': 5531204,
  'location_dw_end': 5532204},
 {'name': 'SCO5089',
  'up_repair': Amplicon(1000),
  'up_forwar_p': f1000 14-mer:5'-CCAGGACGCGAAGG-3',
  'up_reverse_p': r1000 18-mer:5'-GGCTTTCTCCAGTTCTCG-3',
  'tm_up_forwar_p': 54,
  'tm_up_reverse_p': 56,
  'ta_up': 58,
  'location_up_start': 5531448,
  'location_up_end': 5532448,
  'dw_repair': Amplicon(1000),
  'dw_forwar_p': f1000 13-mer:5'-GCCGGCCGGGAGA-3',
  'dw_reverse_p': r1000 14-mer:5'-GGGGGCCTGCTCGT-3',
  'tm_dw_forwar_p': 59,
 

In [10]:

# Digest the plasmids
processed_records = [Dseqrecord(record, circular=True).cut(StuI)[0] for record in sgRNA_vectors]

# Rename them appropriately
for i in range(len(processed_records)):
    processed_records[i].name = sgRNA_vectors[i].name

# Assembly 
##### THE PROBLEM IS HERE
assembly_data = assemble_multiple_plasmids_with_repair_templates_for_deletion(genes_to_KO, processed_records, 
                                                                              repair_templates_data, 
                                                                              overlap=40)
# updating the primer names to something systematic.
update_primer_names(assembly_data)
print(assembly_data)

# Parse through the primer df
primer_df = create_primer_df_from_dict(assembly_data)

primer_df

[{'gene_name': 'SCO5087', 'name': 'CRISPR-Cas9_plasmid_targeting_SCO5087_position_271_5', 'contig': Dseqrecord(o13279), 'up_forwar_p_name': 'primer_1', 'up_reverse_p_name': 'primer_2', 'up_forwar_p': primer_1 56-mer:5'-AAGGCCGCTTTTGCG..CAG-3', 'up_reverse_p': primer_2 36-mer:5'-GACTCCGGTGATGAG..GAC-3', 'up_forwar_primer_str': 'AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGACGCCGGAAATCCAG', 'up_reverse_primer_str': 'GACTCCGGTGATGAGGACGCCCCATCTCCCTTCGAC', 'up_forwar_p_anneal': Seq('GACGCCGGAAATCCAG'), 'up_reverse_p_anneal': Seq('CCCATCTCCCTTCGAC'), 'tm_up_forwar_p': 56, 'tm_up_reverse_p': 54, 'ta_up': 58, 'dw_forwar_p_name': 'primer_3', 'dw_reverse_p_name': 'primer_4', 'dw_forwar_p': primer_3 36-mer:5'-GGCGGTCGAAGGGAG..CGG-3', 'dw_reverse_p': primer_4 55-mer:5'-TCCCCGTCCGGGACC..CCC-3', 'dw_forwar_primer_str': 'GGCGGTCGAAGGGAGATGGGGCGTCCTCATCACCGG', 'dw_reverse_primer_str': 'TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGAGTAGAGGCGGCCC', 'dw_forwar_p_anneal': Seq('GCGTCCTCATCACCGG'), 'dw_reverse_p_a

,template,direction,f_primer_anneal(5-3),r_primer_anneal(5-3),f_tm,r_tm,ta,f_primer_sequences(5-3),r_primer_sequences(5-3),f_primer_name,r_primer_name
0,SCO5087,upstream,GACGCCGGAAATCCAG,CCCATCTCCCTTCGAC,56,54,58,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGACGCC...,GACTCCGGTGATGAGGACGCCCCATCTCCCTTCGAC,primer_1,primer_2
1,SCO5087,downstream,GCGTCCTCATCACCGG,GAGTAGAGGCGGCCC,58,57,61,GGCGGTCGAAGGGAGATGGGGCGTCCTCATCACCGG,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGAGTAG...,primer_3,primer_4
2,SCO5087,upstream,GACGCCGGAAATCCAG,CCCATCTCCCTTCGAC,56,54,58,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGACGCC...,GACTCCGGTGATGAGGACGCCCCATCTCCCTTCGAC,primer_1,primer_2
3,SCO5087,downstream,GCGTCCTCATCACCGG,GAGTAGAGGCGGCCC,58,57,61,GGCGGTCGAAGGGAGATGGGGCGTCCTCATCACCGG,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGAGTAG...,primer_3,primer_4
4,SCO5089,upstream,CCAGGACGCGAAGG,GGCTTTCTCCAGTTCTCG,54,56,58,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGCCAGGA...,CCGGTGTTCTCCCGGCCGGCGGCTTTCTCCAGTTCTCG,primer_5,primer_6
5,SCO5089,downstream,GCCGGCCGGGAGA,GGGGGCCTGCTCGT,59,60,63,CCCGAGAACTGGAGAAAGCCGCCGGCCGGGAGA,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGGGGGC...,primer_7,primer_8
6,SCO5089,upstream,CCAGGACGCGAAGG,GGCTTTCTCCAGTTCTCG,54,56,58,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGCCAGGA...,CCGGTGTTCTCCCGGCCGGCGGCTTTCTCCAGTTCTCG,primer_5,primer_6
7,SCO5089,downstream,GCCGGCCGGGAGA,GGGGGCCTGCTCGT,59,60,63,CCCGAGAACTGGAGAAAGCCGCCGGCCGGGAGA,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGGGGGC...,primer_7,primer_8
8,SCO5090,upstream,GCCACGCCCGCCG,GCCGCCTCGGCCAGT,63,64,66,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGCCACG...,GACCTCACGGACCTCGACCGGCCGCCTCGGCCAGT,primer_9,primer_10
9,SCO5090,downstream,CGGTCGAGGTCCGT,ACCGTGCGGGACT,56,55,59,GGCGCACTGGCCGAGGCGGCCGGTCGAGGTCCGT,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGACCGTG...,primer_11,primer_12


In [11]:
#Find_unique primers
unique_df = primer_df.drop_duplicates(keep='first')
unique_df

,template,direction,f_primer_anneal(5-3),r_primer_anneal(5-3),f_tm,r_tm,ta,f_primer_sequences(5-3),r_primer_sequences(5-3),f_primer_name,r_primer_name
0,SCO5087,upstream,GACGCCGGAAATCCAG,CCCATCTCCCTTCGAC,56,54,58,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGACGCC...,GACTCCGGTGATGAGGACGCCCCATCTCCCTTCGAC,primer_1,primer_2
1,SCO5087,downstream,GCGTCCTCATCACCGG,GAGTAGAGGCGGCCC,58,57,61,GGCGGTCGAAGGGAGATGGGGCGTCCTCATCACCGG,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGAGTAG...,primer_3,primer_4
4,SCO5089,upstream,CCAGGACGCGAAGG,GGCTTTCTCCAGTTCTCG,54,56,58,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGCCAGGA...,CCGGTGTTCTCCCGGCCGGCGGCTTTCTCCAGTTCTCG,primer_5,primer_6
5,SCO5089,downstream,GCCGGCCGGGAGA,GGGGGCCTGCTCGT,59,60,63,CCCGAGAACTGGAGAAAGCCGCCGGCCGGGAGA,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGGGGGC...,primer_7,primer_8
8,SCO5090,upstream,GCCACGCCCGCCG,GCCGCCTCGGCCAGT,63,64,66,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGCCACG...,GACCTCACGGACCTCGACCGGCCGCCTCGGCCAGT,primer_9,primer_10
9,SCO5090,downstream,CGGTCGAGGTCCGT,ACCGTGCGGGACT,56,55,59,GGCGCACTGGCCGAGGCGGCCGGTCGAGGTCCGT,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGACCGTG...,primer_11,primer_12


In [12]:
idt_df = create_idt_order_dataframe(unique_df, concentration="25nm", purification="STD")
idt_df

,Name,Sequence,Concentration,Purification
0,primer_1,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGACGCC...,25nm,STD
1,primer_3,GGCGGTCGAAGGGAGATGGGGCGTCCTCATCACCGG,25nm,STD
2,primer_5,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGCCAGGA...,25nm,STD
3,primer_7,CCCGAGAACTGGAGAAAGCCGCCGGCCGGGAGA,25nm,STD
4,primer_9,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGCCACG...,25nm,STD
5,primer_11,GGCGCACTGGCCGAGGCGGCCGGTCGAGGTCCGT,25nm,STD
6,primer_2,GACTCCGGTGATGAGGACGCCCCATCTCCCTTCGAC,25nm,STD
7,primer_4,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGAGTAG...,25nm,STD
8,primer_6,CCGGTGTTCTCCCGGCCGGCGGCTTTCTCCAGTTCTCG,25nm,STD
9,primer_8,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGGGGGC...,25nm,STD


In [13]:
# Contigs
assembled_contigs = []
for data in assembly_data: 
    contig_record = data['contig']
    contig_record.id = f"{data['name']}_w_rep"
    assembled_contigs.append(contig_record)
assembled_contigs


[Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279)]

In [14]:
assembled_contigs

[Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279),
 Dseqrecord(o13279)]

In [15]:
print_plasmids = False

if print_plasmids:
    # Write to genbank files
    output_directory = "../../data/plasmids/sgRNA_plasmids_pCRISPR-Cas9_with_repair_templates/"
    for contig in assembled_contigs: 
        contig.write(f"{output_directory}/{contig.id}.gb")

In [16]:
# Getting checking primers
checking_primers_df = checking_primers(path_to_genome, genes_to_KO, 
                                       flanking_region=flanking_region,
                                       target_tm = melting_temperature, 
                                        primer_concentration = primer_concentration, 
                                        polymerase = chosen_polymerase)
checking_primers_df


,Locus Tag,f_primer_name,r_primer_name,f_primer_sequences(5-3),r_primer_sequences(5-3),f_tm,r_tm,ta
0,SCO5087,SCO5087_fwd_checking_primer,SCO5087_rev_checking_primer,GACGATTCGGCCCGTG,CAGGGCGTCCAGGC,59,58,61
1,SCO5089,SCO5089_fwd_checking_primer,SCO5089_rev_checking_primer,CGTACGGCGAACTCGC,GTCCCCGGACGGC,59,57,60
2,SCO5090,SCO5090_fwd_checking_primer,SCO5090_rev_checking_primer,GGCGGCGGCCC,GACGTCCCCCGCGAAC,57,62,61


In [17]:
# making the primers into dseqrecords
checking_primers_df_idt = create_idt_order_dataframe(checking_primers_df)
checking_primers_df_idt


,Name,Sequence,Concentration,Purification
0,SCO5087_fwd_checking_primer,GACGATTCGGCCCGTG,25nm,STD
1,SCO5089_fwd_checking_primer,CGTACGGCGAACTCGC,25nm,STD
2,SCO5090_fwd_checking_primer,GGCGGCGGCCC,25nm,STD
3,SCO5087_rev_checking_primer,CAGGGCGTCCAGGC,25nm,STD
4,SCO5089_rev_checking_primer,GTCCCCGGACGGC,25nm,STD
5,SCO5090_rev_checking_primer,GACGTCCCCCGCGAAC,25nm,STD


In [18]:

full_idt = pd.concat([idt_df, checking_primers_df_idt])
full_idt

,Name,Sequence,Concentration,Purification
0,primer_1,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGACGCC...,25nm,STD
1,primer_3,GGCGGTCGAAGGGAGATGGGGCGTCCTCATCACCGG,25nm,STD
2,primer_5,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGCCAGGA...,25nm,STD
3,primer_7,CCCGAGAACTGGAGAAAGCCGCCGGCCGGGAGA,25nm,STD
4,primer_9,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGCCACG...,25nm,STD
5,primer_11,GGCGCACTGGCCGAGGCGGCCGGTCGAGGTCCGT,25nm,STD
6,primer_2,GACTCCGGTGATGAGGACGCCCCATCTCCCTTCGAC,25nm,STD
7,primer_4,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGAGTAG...,25nm,STD
8,primer_6,CCGGTGTTCTCCCGGCCGGCGGCTTTCTCCAGTTCTCG,25nm,STD
9,primer_8,TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGGGGGGC...,25nm,STD


In [ ]:


input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

output_files = [
    {"name": "cBEST_w_sgRNAs.gb", "content": assembled_contigs}, # LIST OF Dseqrecords
    {"name": "primer_df.csv", "content": primer_df},
    {"name": "full_idt.csv", "content": full_idt},
    {"name": "sgrna_df.csv", "content": sgrna_df},
    {"name": "filtered_df.csv", "content": filtered_df}
]

input_values = {
    "genes_to_knockout": genes_to_KO,


    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group,

    },  
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region": flanking_region
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

# Data and time
timestamp = datetime.utcnow().isoformat()

generate_project_directory_structure(
    project_name=f"CRISPR_cas9_inframe_deletion_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths

)